# CIML Predictions

This notebook we train and evaluate CIML experiments using the functions `gather_results` and `tf_trainer functions` of the [ciml project](https://github.com/mtreinish/ciml).

In [187]:
from ciml import gather_results
from ciml import tf_trainer
import numpy as np
import pandas as pd
import collections

First we define the data path, dataset and experiment to gather the right input dataset for and the configuration for the experiment.

In [162]:
data_path = '/Users/kw/ciml_data/cimlodsceu2019seed'
dataset = 'usr_1m-1min-node_provider'
#dataset = 'usr_1m-1min-status'
experiment = 'dnn-3x100-500epochs-bs128'
#experiment = 'dnn-5x100-500epochs-bs128'

In [163]:
experiment_data = gather_results.load_experiment(
        experiment, data_path=data_path)
dataset_data = gather_results.load_model_config(
        dataset, data_path=data_path)

In [192]:
dataset_data

{'build_name': 'tempest-full',
 'sample_interval': '1min',
 'features_regex': '(usr|1m)',
 'class_label': 'node_provider',
 'aggregation_functions': [],
 'training_set': 0.6,
 'dev_set': 0.2,
 'test_set': 0.2,
 'normalized_length': 92,
 'labels': ['usr0',
  'usr1',
  'usr2',
  'usr3',
  'usr4',
  'usr5',
  'usr6',
  'usr7',
  'usr8',
  'usr9',
  'usr10',
  'usr11',
  'usr12',
  'usr13',
  'usr14',
  'usr15',
  'usr16',
  'usr17',
  'usr18',
  'usr19',
  'usr20',
  'usr21',
  'usr22',
  'usr23',
  'usr24',
  'usr25',
  'usr26',
  'usr27',
  'usr28',
  'usr29',
  'usr30',
  'usr31',
  'usr32',
  'usr33',
  'usr34',
  'usr35',
  'usr36',
  'usr37',
  'usr38',
  'usr39',
  'usr40',
  'usr41',
  'usr42',
  'usr43',
  'usr44',
  'usr45',
  'usr46',
  'usr47',
  'usr48',
  'usr49',
  'usr50',
  'usr51',
  'usr52',
  'usr53',
  'usr54',
  'usr55',
  'usr56',
  'usr57',
  'usr58',
  'usr59',
  'usr60',
  'usr61',
  'usr62',
  'usr63',
  'usr64',
  'usr65',
  'usr66',
  'usr67',
  'usr68',
  'us

In [165]:
experiment_data

{'estimator': 'tf.estimator.DNNClassifier',
 'params': {},
 'hyper_params': {'steps': 9500,
  'batch_size': '128',
  'epochs': '500',
  'hidden_units': [100, 100, 100],
  'optimizer': 'Adagrad',
  'learning_rate': 0.05}}

experiment_data

In [166]:
estimator = experiment_data['estimator']
hyper_params = experiment_data['hyper_params']
params = experiment_data['params']
steps = int(hyper_params['steps'])
num_epochs = int(hyper_params['epochs'])
batch_size = int(hyper_params['batch_size'])
optimizer = hyper_params['optimizer']
learning_rate = float(hyper_params['learning_rate'])
class_label = dataset_data['class_label']
labels = gather_results.load_dataset(dataset, 'labels', data_path=data_path)['labels']
training_data = gather_results.load_dataset(dataset, 'training', data_path=data_path)
test_data = gather_results.load_dataset(dataset, 'test', data_path=data_path)

#label_vocabulary = None
if class_label == 'node_provider':
    label_vocabulary = set(['rax', 'ovh', 'packethost-us-west-1',
                            'vexxhost', 'limestone-regionone',
                            'inap-mtl01', 'fortnebula-regionone'])
elif class_label == 'node_provider_all':
    label_vocabulary = set(['rax-iad', 'ovh-bhs1', 'packethost-us-west-1',
                            'rax-dfw', 'vexxhost-ca-ymq-1', 'ovh-gra1',
                            'limestone-regionone', 'inap-mtl01', 'rax-ord',
                            'vexxhost-sjc1', 'fortnebula-regionone'])
else:
    label_vocabulary = None

model_dir = gather_results.get_model_folder(dataset, experiment, data_path=data_path)


In [167]:
model_dir

'/Users/kw/ciml_data/cimlodsceu2019seed/usr_1m-1min-node_provider/dnn-3x100-500epochs-bs128'

In [168]:
estimator = tf_trainer.get_estimator(
        estimator, hyper_params, params, labels, model_dir,
        optimizer=adagrad.AdagradOptimizer(learning_rate=0.05),
        label_vocabulary=label_vocabulary, gpu=False)

INFO:tensorflow:Using config: {'_model_dir': '/Users/kw/ciml_data/cimlodsceu2019seed/usr_1m-1min-node_provider/dnn-3x100-500epochs-bs128', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': None, '_save_checkpoints_secs': 300, '_session_config': allow_soft_placement: true
, '_keep_checkpoint_max': 10, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_train_distribute': None, '_device_fn': None, '_protocol': None, '_eval_distribute': None, '_experimental_distribute': None, '_experimental_max_worker_delay_secs': None, '_session_creation_timeout_secs': 7200, '_service': None, '_cluster_spec': ClusterSpec({}), '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_evaluation_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}


In [169]:
input_fn=tf_trainer.get_input_fn(shuffle=True,
                    batch_size=batch_size, num_epochs=num_epochs,
                    labels=labels, **training_data)

In [170]:
"""inputs = []
while True:
    input = input_fn()
    if not input:
        break
    else:
        inputs.append(input)"""

'inputs = []\nwhile True:\n    input = input_fn()\n    if not input:\n        break\n    else:\n        inputs.append(input)'

In [171]:
data_path

'/Users/kw/ciml_data/cimlodsceu2019seed'

In [172]:
training_result = tf_trainer.get_training_method(estimator)(
                    input_fn=tf_trainer.get_input_fn(shuffle=True,
                    batch_size=batch_size, num_epochs=num_epochs,
                    labels=labels, **training_data), steps=steps)

INFO:tensorflow:Calling model_fn.

If you intended to run this layer in float32, you can safely ignore this warning. If in doubt, this warning is likely only an issue if you are porting a TensorFlow 1.X model to TensorFlow 2.

To change all layers to have dtype float64 by default, call `tf.keras.backend.set_floatx('float64')`. To change just this layer, pass dtype='float64' to the layer constructor. If you are the author of this layer, you can disable autocasting by passing autocast=False to the base Layer constructor.

INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from /Users/kw/ciml_data/cimlodsceu2019seed/usr_1m-1min-node_provider/dnn-3x100-500epochs-bs128/model.ckpt-9286
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Calling checkpoint listeners before saving checkpoint 9286...
INFO:tensorflow:Saving checkpoints for 9286 

INFO:tensorflow:global_step/sec: 28.0718
INFO:tensorflow:loss = 0.001494943, step = 15986 (3.556 sec)
INFO:tensorflow:global_step/sec: 30.1745
INFO:tensorflow:loss = 0.001242709, step = 16086 (3.314 sec)
INFO:tensorflow:global_step/sec: 30.7769
INFO:tensorflow:loss = 0.0006756574, step = 16186 (3.251 sec)
INFO:tensorflow:global_step/sec: 30.6173
INFO:tensorflow:loss = 0.0018570684, step = 16286 (3.265 sec)
INFO:tensorflow:global_step/sec: 29.7428
INFO:tensorflow:loss = 0.0011542242, step = 16386 (3.374 sec)
INFO:tensorflow:global_step/sec: 28.7687
INFO:tensorflow:loss = 0.0013075632, step = 16486 (3.465 sec)
INFO:tensorflow:global_step/sec: 29.3181
INFO:tensorflow:loss = 0.0014361411, step = 16586 (3.409 sec)
INFO:tensorflow:global_step/sec: 25.9928
INFO:tensorflow:loss = 0.0015845525, step = 16686 (3.849 sec)
INFO:tensorflow:global_step/sec: 26.3222
INFO:tensorflow:loss = 0.0010777276, step = 16786 (3.798 sec)
INFO:tensorflow:global_step/sec: 30.0265
INFO:tensorflow:loss = 0.001749295

In [173]:
eval_data = gather_results.load_dataset(dataset, 'test', data_path=data_path)

In [174]:
eval_size = len(eval_data['example_ids'])
eval_data.keys()

dict_keys(['examples', 'example_ids', 'classes'])

In [175]:
prediction = estimator.predict(input_fn=tf_trainer.get_input_fn(
                                batch_size=eval_size, num_epochs=1,
                                labels=labels, **eval_data))

In [176]:
predictions = [x for x in prediction]

INFO:tensorflow:Calling model_fn.

If you intended to run this layer in float32, you can safely ignore this warning. If in doubt, this warning is likely only an issue if you are porting a TensorFlow 1.X model to TensorFlow 2.

To change all layers to have dtype float64 by default, call `tf.keras.backend.set_floatx('float64')`. To change just this layer, pass dtype='float64' to the layer constructor. If you are the author of this layer, you can disable autocasting by passing autocast=False to the base Layer constructor.

INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from /Users/kw/ciml_data/cimlodsceu2019seed/usr_1m-1min-node_provider/dnn-3x100-500epochs-bs128/model.ckpt-18572
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.


In [177]:
predictions

[{'logits': array([ -8.928443  ,   0.90596014, -14.328681  ,  14.876499  ,
          -7.179666  ,  -1.0007036 ,  -4.7466745 ], dtype=float32),
  'probabilities': array([4.5882190e-11, 8.5638993e-07, 2.0718130e-13, 9.9999905e-01,
         2.6371119e-10, 1.2723832e-07, 3.0044427e-09], dtype=float32),
  'class_ids': array([3]),
  'classes': array([b'ovh'], dtype=object),
  'all_class_ids': array([0, 1, 2, 3, 4, 5, 6], dtype=int32),
  'all_classes': array([b'fortnebula-regionone', b'inap-mtl01', b'vexxhost', b'ovh',
         b'limestone-regionone', b'rax', b'packethost-us-west-1'],
        dtype=object)},
 {'logits': array([ -2.0642838,  -8.038662 ,   3.29522  ,  -0.1434648,  -9.003074 ,
           5.6692057, -28.137304 ], dtype=float32),
  'probabilities': array([3.9936003e-04, 1.0156057e-06, 8.4911697e-02, 2.7262496e-03,
         3.8715640e-07, 9.1196132e-01, 1.8966853e-15], dtype=float32),
  'class_ids': array([5]),
  'classes': array([b'rax'], dtype=object),
  'all_class_ids': array([0

In [178]:
p_classes = [x["class_ids"][0] for x in predictions]
actual_classes = eval_data["classes"]
classes = zip(p_classes, actual_classes)
counter = collections.Counter(classes)
sorted(counter.values(), reverse=True)[:15]

[337, 196, 107, 26, 22, 15, 12, 12, 7, 7, 6, 6, 4, 4, 4]

In [179]:
counter

Counter({(3, 'limestone-regionone'): 12,
         (5, 'rax'): 337,
         (3, 'ovh'): 196,
         (1, 'inap-mtl01'): 107,
         (6, 'ovh'): 4,
         (4, 'ovh'): 15,
         (5, 'fortnebula-regionone'): 3,
         (2, 'vexxhost'): 26,
         (3, 'fortnebula-regionone'): 1,
         (4, 'limestone-regionone'): 12,
         (3, 'vexxhost'): 22,
         (6, 'packethost-us-west-1'): 4,
         (2, 'inap-mtl01'): 2,
         (3, 'packethost-us-west-1'): 7,
         (3, 'inap-mtl01'): 6,
         (6, 'inap-mtl01'): 1,
         (2, 'ovh'): 6,
         (1, 'limestone-regionone'): 1,
         (1, 'ovh'): 4,
         (5, 'packethost-us-west-1'): 1,
         (2, 'limestone-regionone'): 4,
         (1, 'rax'): 2,
         (6, 'vexxhost'): 1,
         (5, 'inap-mtl01'): 7,
         (6, 'limestone-regionone'): 3,
         (0, 'rax'): 1,
         (0, 'fortnebula-regionone'): 1,
         (4, 'vexxhost'): 2,
         (2, 'packethost-us-west-1'): 2,
         (4, 'inap-mtl01'): 1,
        

In [180]:
serializable_pred

[{'logits': array([ -8.5038185 ,   1.0745989 , -13.41295   ,  14.034415  ,
          -6.611523  ,  -0.87847054,  -4.2996163 ], dtype=float32),
  'probabilities': array([1.6284253e-10, 2.3530017e-06, 1.2015995e-12, 9.9999726e-01,
         1.0803922e-09, 3.3374508e-07, 1.0905104e-08], dtype=float32),
  'class_ids': array([3]),
  'classes': ['ovh'],
  'all_class_ids': array([0, 1, 2, 3, 4, 5, 6], dtype=int32),
  'all_classes': ['fortnebula-regionone',
   'inap-mtl01',
   'vexxhost',
   'ovh',
   'limestone-regionone',
   'rax',
   'packethost-us-west-1']},
 {'logits': array([ -1.9330785 ,  -7.508508  ,   3.0481033 ,  -0.03661131,
          -8.272822  ,   5.486122  , -26.74158   ], dtype=float32),
  'probabilities': array([5.4914446e-04, 2.0811729e-06, 7.9980940e-02, 3.6585757e-03,
         9.6910514e-07, 9.1580832e-01, 9.2361845e-15], dtype=float32),
  'class_ids': array([5]),
  'classes': ['rax'],
  'all_class_ids': array([0, 1, 2, 3, 4, 5, 6], dtype=int32),
  'all_classes': ['fortnebula

In [181]:
serializable_pred = []
for pred in predictions:
    _classes = pred['classes']
    _all_classes = pred['all_classes']
    pred['classes'] = [x.decode("utf-8") for x in _classes]
    pred['all_classes'] = [x.decode("utf-8") for x in _all_classes]
    serializable_pred.append(pred)

prediction_name = "prediction_" + dataset
pred_data = zip(eval_data['example_ids'], serializable_pred,
                eval_data['classes'])
gather_results.save_data_json(
    dataset, [x for x in pred_data],
    prediction_name, sub_folder=experiment, data_path=data_path)